In [3]:
import os
os.listdir("/content")


['.config', 'fear_greed_index.csv', 'historical_data.csv', 'sample_data']

In [4]:
import pandas as pd

trades_raw = pd.read_csv("/content/historical_data.csv")
sentiment = pd.read_csv("/content/fear_greed_index.csv")

print(trades_raw.shape)
print(sentiment.shape)


(37109, 16)
(2644, 4)


/tmp/ipython-input-2827581016.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  trades_raw = pd.read_csv("/content/historical_data.csv")


In [5]:
trades_raw["Timestamp IST"] = pd.to_datetime(
    trades_raw["Timestamp IST"], errors="coerce"
)

trades_raw["date"] = trades_raw["Timestamp IST"].dt.date
trades_raw = trades_raw.dropna(subset=["date"])


In [6]:
trades = trades_raw.rename(columns={
    "Account": "account",
    "Coin": "symbol",
    "Side": "side",
    "Size USD": "size_usd",
    "Closed PnL": "closedPnL"
})

trades = trades[
    ["account", "symbol", "side", "size_usd", "closedPnL", "date"]
]

sentiment = sentiment.rename(columns={
    "classification": "Classification"
})

sentiment = sentiment[["date", "Classification"]]


In [7]:
trades["date"] = pd.to_datetime(trades["date"])
sentiment["date"] = pd.to_datetime(sentiment["date"])

trades = trades.sort_values("date")
sentiment = sentiment.sort_values("date")


In [8]:
merged = pd.merge_asof(
    trades,
    sentiment,
    on="date",
    direction="backward"
)

print(merged.shape)
merged.head()


(13615, 7)


,account,symbol,side,size_usd,closedPnL,date,Classification
0,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,SELL,551.09,16.58460,2024-01-03,Greed
1,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,SELL,124.05,3.21024,2024-01-03,Greed
2,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,SELL,1045.04,27.04384,2024-01-03,Greed
3,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,SELL,357.59,9.25376,2024-01-03,Greed
4,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,SELL,2006.43,51.92320,2024-01-03,Greed


In [9]:
merged["is_profitable"] = merged["closedPnL"] > 0
merged["abs_pnl"] = merged["closedPnL"].abs()
merged["trade_count"] = 1


In [10]:
performance_summary = merged.groupby("Classification").agg(
    avg_pnl=("closedPnL", "mean"),
    median_pnl=("closedPnL", "median"),
    win_rate=("is_profitable", "mean"),
    avg_loss=("abs_pnl", "mean"),
    trades=("trade_count", "sum")
)

performance_summary


,avg_pnl,median_pnl,win_rate,avg_loss,trades
Classification,,,,,
Extreme Fear,24.873288,0.000000,0.233871,24.873288,124
Extreme Greed,-39.149743,0.000000,0.320570,159.735018,1123
Fear,359.091611,0.125800,0.507880,422.075706,3680
Greed,81.238769,0.000000,0.440560,172.446803,8210
Neutral,89.284692,11.755104,0.774059,89.284692,478


In [11]:
pnl_per_trade = (
    merged.groupby("Classification")["closedPnL"].sum() /
    merged.groupby("Classification")["trade_count"].sum()
)
pnl_per_trade


,0
Classification,
Extreme Fear,24.873288
Extreme Greed,-39.149743
Fear,359.091611
Greed,81.238769
Neutral,89.284692


In [12]:
risk_return = (
    merged.groupby("Classification")["closedPnL"].mean() /
    merged.groupby("Classification")["abs_pnl"].mean()
)
risk_return


,0
Classification,
Extreme Fear,1.000000
Extreme Greed,-0.245092
Fear,0.850775
Greed,0.471095
Neutral,1.000000


In [13]:
print(merged.shape)


(13615, 10)
